In [103]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot, plot
import plotly as py
init_notebook_mode(connected=True)
from wordcloud import WordCloud
import matplotlib.pyplot as plt

from sklearn.datasets import load_iris, load_boston
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")



ADECUACIÓN DE MI DATAFRAME

In [104]:
#NETFLIX
#Creado en 2007
#Dataset "adecuado" a partir del 2018

dfnetflix=pd.read_csv("C:/Users/Usuario/Desktop/EDA/data/Datasets/netflix_titles.csv")

#Adecuo mi df para que solo me enseñe lo que necesito
netflix=dfnetflix[["type","title","date_added"]]
netflix=netflix[netflix["date_added"].isnull()==False] #10 values NaN que omito (de 8k)

#Transformo el tipo de la columna fecha en tipo datetime
netflix.date_added=pd.to_datetime(netflix.date_added) 

#Quitamos valores que no estan correctamente administrados (7k valores)
netflix=netflix.loc[netflix["date_added"]>"2018"]

print(netflix.shape)
netflix.head(6)


(6987, 3)


,type,title,date_added
0,Movie,Dick Johnson Is Dead,2021-09-25
1,TV Show,Blood & Water,2021-09-24
2,TV Show,Ganglands,2021-09-24
3,TV Show,Jailbirds New Orleans,2021-09-24
4,TV Show,Kota Factory,2021-09-24
5,TV Show,Midnight Mass,2021-09-24


MERGE CON TOP 250

In [164]:
#Para evaluar cuantas peliculas o series son top, haremos un merge con el dataframe que hemos obtenido  mediante web scrapping.

IMDBP=pd.read_csv("C:/Users/Usuario/Desktop/EDA/data/Datasets/IMDBP", sep= "\t", index_col=0)

IMDBS=pd.read_csv("C:/Users/Usuario/Desktop/EDA/data/Datasets/IMDBS", sep= "\t",index_col=0)

Net_=pd.merge(netflix,IMDBP, how="outer")
flix_=pd.merge(netflix,IMDBS, how="outer") 

#Quito los valores del top250 que no están en Netflix (no tienen date_added)
Netflix_=flix_[flix_["date_added"].notna()]


Netflix_

,type,title,date_added,Rates,NVotes
0,Movie,Dick Johnson Is Dead,2021-09-25,NaN,NaN
1,TV Show,Blood & Water,2021-09-24,NaN,NaN
2,TV Show,Ganglands,2021-09-24,NaN,NaN
3,TV Show,Jailbirds New Orleans,2021-09-24,NaN,NaN
4,TV Show,Kota Factory,2021-09-24,8.69,71680.0
...,...,...,...,...,...
6982,Movie,Zodiac,2019-11-20,NaN,NaN
6983,TV Show,Zombie Dumb,2019-07-01,NaN,NaN
6984,Movie,Zombieland,2019-11-01,NaN,NaN
6985,Movie,Zoom,2020-01-11,NaN,NaN


PREPARACIÓN PARA UTILIZAR MI DF

In [131]:
#Para ser mas utilitario, me quedo con los titulos sacados en los ULTIMOS 15 dias (acumulando cuanto ha sacado HASTA el dia "x")
date15nethistoric=Netflix_.groupby(pd.Grouper(key="date_added", freq="SMS"))


#Creo el grafico historico de Netflix
df=date15nethistoric.count()

trace1 = go.Scatter(
                    x = df.index,
                    y = df['type'],
                    name = 'Netflix',
                    mode= 'lines',
                    marker = dict(color = 'rgba(200, 2, 2, 0.8)'))
fig=go.Figure(trace1)
iplot(fig)

In [153]:
#Saco la cantidad de peliculas cada 15 dias que ha lanzado Netflix

Netflixf=Netflix_[Netflix_["type"]=="Movie"]

groupfilm=Netflixf.groupby(pd.Grouper(key="date_added", freq="SMS"))
gf=groupfilm.count()

filmbar = go.Bar(
                    x = gf.index,
                    y = gf['type'],
                    name = 'Netflix',
                  
                    marker = dict(color = 'rgba(200, 2, 2, 0.8)'))
filmfig=go.Figure(filmbar)
iplot(filmfig)

In [159]:
#Saco la cantidad de series cada 15 dias que ha lanzado Netflix
Netflixf=Netflix_[Netflix_["type"]=="TV Show"]

groupfilm=Netflixf.groupby(pd.Grouper(key="date_added", freq="SMS"))
gf=groupfilm.count()

filmbar = go.Bar(
                    x = gf.index,
                    y = gf['type'],
                    name = 'Netflix',
                  
                    marker = dict(color = 'rgba(200, 2, 2, 0.8)'))
filmfig=go.Figure(filmbar)
iplot(filmfig)

In [144]:
#Para poder comparar Netflix con las otras plataformas, busco un tipo de grafica que sea mas visual las tendencias en los lanzamientos

netflix21=Netflix_.loc[Netflix_["date_added"]>"2020"]

date15_21net=netflix21.groupby(pd.Grouper(key="date_added", freq="SMS"))

df=date15_21net.count()

trace1 = go.Scatter(
                    x = df.index,
                    y = df['title'],
                    name = 'Netflix',
                    mode= 'lines',
                    marker = dict(color = 'rgba(200, 2, 2, 0.8)'))
fig=go.Figure(trace1)
iplot(fig)



In [108]:
#Descarto el dataframe de amazon porque no me aporta la información que requiero

dfamazon=pd.read_csv("C:/Users/Usuario/Desktop/EDA/data/Datasets/amazon_prime_titles.csv")
amazon=dfamazon[["type","title","date_added"]]
amazon=amazon[amazon["date_added"].isnull()]  #Tiene muchos valores NaN (prescindo)
amazon


,type,title,date_added
16,Movie,Zoombies,NaN
17,TV Show,Zoo Babies,NaN
18,TV Show,Zoë Coombs Marr: Bossy Bottom,NaN
19,Movie,Zoe,NaN
20,TV Show,Zoboomafoo,NaN
...,...,...,...
9663,Movie,Pride Of The Bowery,NaN
9664,TV Show,Planet Patrol,NaN
9665,Movie,Outpost,NaN
9666,TV Show,Maradona: Blessed Dream,NaN


In [109]:
#Realizo el mismo proceso de limpieza y lo mergeo con los dataframes que he sacado, exactamente igual que con netflix

dfhulu=pd.read_csv("C:/Users/Usuario/Desktop/EDA/data/Datasets/hulu_titles.csv")
hulu=dfhulu[["type","title","date_added"]]
hulu=hulu[hulu["date_added"].isnull()==False] #30 values NaN
hulu.date_added=pd.to_datetime(hulu.date_added)
hulu=hulu.loc[hulu["date_added"]>"2020"]
hu_=pd.merge(hulu,IMDBP, how="outer")
lu_=pd.merge(hulu,IMDBS, how="outer")
Hulu_=lu_[lu_["date_added"].notna()]


date15hul=hulu.groupby(pd.Grouper(key="date_added", freq="SM"))



In [110]:
date15hul=Hulu_.groupby(pd.Grouper(key="date_added", freq="SMS"))


#Creo el grafico historico de Hulu
dfe=date15hul.count()

trace2 = go.Scatter(
                    x = dfe.index,
                    y = dfe['title'],
                    name = 'Hulu',
                    mode= 'lines',
                    marker = dict(color = 'rgba(45, 11, 26, 0.8)'))
fig=go.Figure(trace2)
iplot(fig)

In [111]:
#HULU Y NETFLIX

data=[trace1,trace2]
fig=go.Figure(data=data)
iplot(fig)

In [1]:
#Con Disney realizo el mismo proceso, aunque tengo que indagar un poco mas porque tiene valores no fiables (muchos valores que se han añadido de forma predeterminada en el 1 Nov 2019 ) cuando se lanzó la plataforma

dfdisney=pd.read_csv("C:/Users/Usuario/Desktop/EDA/data/Datasets/disney_plus_titles.csv")
disney=dfdisney[["type","title","date_added"]]
disney=disney[disney["date_added"].isnull()==False] #3 values false
disney.date_added=pd.to_datetime(disney.date_added) 

#Tras analizar suponemos que son la base de las peliculas añadidas durante el año 2019 ya que el dataset empieza a tener una distribución estandarizada a partir del 2020

disney=disney.loc[disney["date_added"]>"2020"]


dis_=pd.merge(disney,IMDBP, how="outer")
ney_=pd.merge(disney,IMDBS, how="outer")

Disney_=ney_[ney_["date_added"].notna()]

Disney_[Disney_["Rates"].notna()]


NameError: name 'pd' is not defined

In [162]:
date15dis=Disney_.groupby(pd.Grouper(key="date_added", freq="SMS"))

#Creo el grafico historico de Disney
dfd=date15dis.count()

trace3 = go.Scatter(
                    x = dfd.index,
                    y = dfd['title'],
                    name = "Disney",
                    mode= 'lines',
                    marker = dict(color = 'rgba(2, 11, 172, 0.8)'))


In [114]:
#HULU NETFLIX DISNEY+
data=[trace1,trace2,trace3]
fig=go.Figure(data=data)
iplot(fig)

In [161]:
#Para sacar la cantidad de peliculas top250 que han lanzado las distinas plataformas, opto por un bar chart
trace11 = go.Bar(
                    x = df.index,
                    y = df['Rates'],
                    name = 'Netflix',
                    
                    marker = dict(color = 'rgba(200, 2, 2, 0.8)'))
trace21 = go.Bar(
                    x = dfe.index,
                    y = dfe['Rates'],
                    name = 'Hulu',
                   
                    marker = dict(color = 'rgba(45, 11, 26, 0.8)'))
trace31 = go.Bar(
                    x = dfd.index,
                    y = dfd['Rates'],
                    name = "Disney",
                   
                    marker = dict(color = 'rgba(2, 11, 172, 0.8)'))

datatop=[trace11,trace21,trace31]
fig=go.Figure(datatop)
iplot(fig)